# Wind量化api for python

1. from WindPy import *：装载WindPy包
2. w.start：启动WindPy
3. w.stop：停止WindPy
4. w.isconnected：判断是否已经登录
5. w.cancelRequest：取消订阅
6. w.wsd：获取历史序列数据
7. w.wsi：获取分钟数据
8. w.wst：获取日内tick级别数据
9. w.wss：获历史截面数据
10. w.wsq：获取和订阅实时行情数据
11. w.wset：获取板块、指数等成分数据
12. w.weqs：获取条件选股结果
13. w.wpf：获取资产管理、组合管理数据

In [9]:
import pandas as pd
from datetime import datetime 

In [2]:
from WindPy import *
w.start() # 启动windpy连接

Welcome to use Wind Quant API for Python (WindPy)!
You can use w.menu to help yourself to create commands(WSD,WSS,WST,WSI,WSQ,...)!

COPYRIGHT (C) 2016 WIND HONGHUI INFORMATION & TECHKNOLEWDGE CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [3]:
w.isconnected() # 检验连接是否成功

True

## 获取全部A股上市公司名称代码

In [4]:
stocks = w.wset("sectorconstituent","date=2016-11-20;sectorid=a001010100000000")

In [5]:
stocks.Fields

['date', 'wind_code', 'sec_name']

In [6]:
df = pd.DataFrame(stocks.Data, index=['uptodate', 'wind_code', 'name'])
df = df.T
df.head()

,uptodate,wind_code,name
0,2016-11-20 00:00:00.005000,000001.SZ,平安银行
1,2016-11-20 00:00:00.005000,000002.SZ,万科A
2,2016-11-20 00:00:00.005000,000004.SZ,国农科技
3,2016-11-20 00:00:00.005000,000005.SZ,世纪星源
4,2016-11-20 00:00:00.005000,000006.SZ,深振业A


## 获取单个股票的历史价格信息，将其写入mongodb

In [50]:
today = datetime.today() # 获取当天的日期
code = '000002.SZ'
_ipo_date = w.wss(code, "ipo_date") # 查询ipo日期
ipo_date = _ipo_date.Data[0][0]
price = w.wsd(code, "open,high,low,close,volume,amt,turn,trade_status", ipo_date, today)
price # price的数据结构

.ErrorCode=0
.Codes=[000002.SZ]
.Fields=[OPEN,HIGH,LOW,CLOSE,VOLUME,AMT,TURN,TRADE_STATUS]
.Times=[19910129,19910130,19910131,19910201,19910204,19910205,19910206,19910207,19910208,19910211,...]
.Data=[[14.58,14.51,14.51,None,14.66,14.73,14.8,14.87,14.94,15.09,...],[14.58,14.51,14.51,None,14.66,14.73,14.8,14.87,14.94,15.09,...],[14.58,14.51,14.51,None,14.66,14.73,14.8,14.87,14.94,15.09,...],[14.58,14.51,14.51,14.51,14.66,14.73,14.8,14.87,14.94,15.09,...],[300.0,1700.0,None,None,5600.0,2900.0,2900.0,4800.0,6600.0,3300.0,...],[22000.0,123000.0,None,None,410000.0,213000.0,215000.0,357000.0,493000.0,249000.0,...],[0.0010714285714285715,0.006071428571428571,None,None,0.02,0.010357142857142856,0.010357142857142856,0.017142857142857144,0.023571428571428573,0.011785714285714287,...],[交易,交易,交易,交易,交易,交易,交易,交易,交易,交易,...]]

In [53]:
pdf = pd.DataFrame(price.Data, index=price.Fields, columns=price.Times).T # 创建股价的DataFrame
pdf.head() # pdf的数据结构

,OPEN,HIGH,LOW,CLOSE,VOLUME,AMT,TURN,TRADE_STATUS
1991-01-29 00:00:00.005,14.58,14.58,14.58,14.58,300,22000,0.00107143,交易
1991-01-30 00:00:00.005,14.51,14.51,14.51,14.51,1700,123000,0.00607143,交易
1991-01-31 00:00:00.005,14.51,14.51,14.51,14.51,None,None,None,交易
1991-02-01 00:00:00.005,None,None,None,14.51,None,None,None,交易
1991-02-04 00:00:00.005,14.66,14.66,14.66,14.66,5600,410000,0.02,交易


In [48]:
# pymongo连接数据库
from pymongo import MongoClient
mongodb_address = 'mongodb://192.168.11.114:27017/'
client = MongoClient(mongodb_address) # 连接
db = client['stockprices'] #获取Database
# collection = db['SZ000002'] #获取Collection

In [29]:
pdf['DATE'] = pdf.index
r = pdf.T.to_dict()
data = [r[i] for i in r]
collection.insert_many(data)

In [54]:
collection.find_one() #测试数据是否插入数据库

{'AMT': 0.0,
 'CLOSE': 14.41,
 'DATE': datetime.datetime(2001, 6, 25, 0, 0, 0, 5000),
 'HIGH': 14.41,
 'LOW': 14.41,
 'OPEN': 14.41,
 'TRADE_STATUS': '停牌一天',
 'TURN': 0.0,
 'VOLUME': 0.0,
 '_id': ObjectId('5831cd2f4255f3bf40da6cc3')}

In [55]:
collection.drop() #删除测试数据

In [42]:
def rename(code):
    l = code.split('.')
    l.reverse()
    s = ''.join(l)
    return s

In [58]:
today = datetime.today() #获取当天的日期
for code in df['wind_code']:
    _ipo_date = w.wss(code, "ipo_date") #查询ipo日期
    ipo_date = _ipo_date.Data[0][0]
    price = w.wsd(code, "open,high,low,close,volume,amt,turn,trade_status", ipo_date, today)
    pdf = pd.DataFrame(price.Data, index=price.Fields, columns=price.Times).T # 创建股价的DataFrame
    collection_name = rename(code)
    collection = db[collection_name] #获取Collection
    pdf['DATE'] = pdf.index #将时间序列插入数据
    r = pdf.T.to_dict() #转换为dict对象
    data = [r[i] for i in r] #生成插入数据的数组
    collection.insert_many(data)
    print('{} collection is inserted'.format(collection_name))
    

000001.SZ collections is inserted
000002.SZ collections is inserted
000004.SZ collections is inserted
000005.SZ collections is inserted
000006.SZ collections is inserted
000007.SZ collections is inserted
000008.SZ collections is inserted
000009.SZ collections is inserted
000010.SZ collections is inserted
000011.SZ collections is inserted
000012.SZ collections is inserted
000014.SZ collections is inserted
000016.SZ collections is inserted
000017.SZ collections is inserted
000018.SZ collections is inserted
000019.SZ collections is inserted
000020.SZ collections is inserted
000021.SZ collections is inserted
000022.SZ collections is inserted
000023.SZ collections is inserted
000025.SZ collections is inserted
000026.SZ collections is inserted
000027.SZ collections is inserted
000028.SZ collections is inserted
000029.SZ collections is inserted
000030.SZ collections is inserted
000031.SZ collections is inserted
000032.SZ collections is inserted
000033.SZ collections is inserted
000034.SZ coll